>Import Libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import cv2
import shutil
import os
from tqdm import tqdm
from glob import glob
from tensorflow.keras.models import load_model
from collections import Counter

>Loading Model

In [2]:
model = load_model("models/DenseNet201_model.h5", compile=False)

In [3]:
class_labels = ['Arson', 'Burglary', 'Explosion', 'Fighting', 'Normal']

### UserInput

In [31]:
user_input = "user_input/Normal_Videos_129_x264_trimmed.mp4"

In [32]:
shutil.rmtree("UserInput_Frames")
os.makedirs("UserInput_Frames", exist_ok=True)

Extracting Frames from User Input

In [33]:
%%time

frame_count = 0

cv_video = cv2.VideoCapture(user_input)

while True:
    status, frame = cv_video.read()

    if status:
        output_filepath = os.path.join("UserInput_Frames", f"{frame_count}_frame.png")
        cv2.imwrite(output_filepath, frame)
        frame_count += 1
    else:
        break
cv_video.release()

CPU times: total: 46.9 ms
Wall time: 288 ms


>Loading Extracted Frames

In [34]:
input_frames_names = glob("UserInput_Frames/*.png")
input_frames_names

['UserInput_Frames\\0_frame.png',
 'UserInput_Frames\\10_frame.png',
 'UserInput_Frames\\11_frame.png',
 'UserInput_Frames\\12_frame.png',
 'UserInput_Frames\\13_frame.png',
 'UserInput_Frames\\14_frame.png',
 'UserInput_Frames\\15_frame.png',
 'UserInput_Frames\\16_frame.png',
 'UserInput_Frames\\17_frame.png',
 'UserInput_Frames\\18_frame.png',
 'UserInput_Frames\\19_frame.png',
 'UserInput_Frames\\1_frame.png',
 'UserInput_Frames\\20_frame.png',
 'UserInput_Frames\\21_frame.png',
 'UserInput_Frames\\22_frame.png',
 'UserInput_Frames\\23_frame.png',
 'UserInput_Frames\\24_frame.png',
 'UserInput_Frames\\25_frame.png',
 'UserInput_Frames\\26_frame.png',
 'UserInput_Frames\\27_frame.png',
 'UserInput_Frames\\28_frame.png',
 'UserInput_Frames\\29_frame.png',
 'UserInput_Frames\\2_frame.png',
 'UserInput_Frames\\30_frame.png',
 'UserInput_Frames\\31_frame.png',
 'UserInput_Frames\\32_frame.png',
 'UserInput_Frames\\33_frame.png',
 'UserInput_Frames\\34_frame.png',
 'UserInput_Frames\\35_

In [35]:
data = []

for filename in tqdm(input_frames_names):
    image = cv2.imread(filename)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (128, 128))
    image = np.array(image)
    image = image/255.0
    data.append(image)

100%|█████████████████████████████████████████████████████████████████████████████████| 90/90 [00:00<00:00, 549.16it/s]


>Model Prediction

In [36]:
prediction = model.predict(np.array(data), batch_size=32, verbose=1)

3/3 [==============================] - 3s 1s/step


In [37]:
predicted_labels = prediction.argmax(axis=1)
predicted_labels = predicted_labels.tolist()
predicted_labels = Counter(predicted_labels)
max_label = max(predicted_labels, key=predicted_labels.get)
class_label = class_labels[max_label]

print(f"class index: {max_label}")
print(f"class label: {class_label}")

class index: 4
class label: Normal


# Thank You 